畫圖.py

In [ ]:
import matplotlib.pyplot as plt
import math

class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y

def distance(city1, city2):
    return math.sqrt((city1.x - city2.x)**2 + (city1.y - city2.y)**2)

nodes = [
    City(60, 200), City(180, 200), City(80, 180), City(140, 180), City(20, 160),
    City(100, 160), City(200, 160), City(140, 140), City(40, 120), City(100, 120)
]

# Calculate distances
distances = {}
for i in range(len(nodes)):
    for j in range(i+1, len(nodes)):
        city1 = nodes[i]
        city2 = nodes[j]
        dist = distance(city1, city2)
        distances[(i, j)] = dist
        distances[(j, i)] = dist

# Plot cities
plt.figure(figsize=(8, 6))
for city in nodes:
    plt.scatter(city.x, city.y, color='blue')
    plt.text(city.x, city.y, f'({city.x}, {city.y})', fontsize=8, ha='right')



plt.title('Cities and Distances')
plt.xlabel('X')
plt.ylabel('Y')
plt.grid(True)
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

暴力破解.py   //execute:40s

In [ ]:
import itertools
import math
import time

class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y

def calculate_distance(city1, city2):
    return math.sqrt((city1.x - city2.x) ** 2 + (city1.y - city2.y) ** 2)

def total_distance(path, cities):
    total = 0
    for i in range(len(path) - 1):
        total += calculate_distance(cities[path[i]], cities[path[i + 1]])
    total += calculate_distance(cities[path[-1]], cities[path[0]])  # Return to the starting point
    return total

def naive_tsp(cities):
    start_time = time.time()
    num_cities = len(cities)
    min_distance = float('inf')
    min_path = []

    perm = list(range(num_cities))

    for p in itertools.permutations(perm):
        distance = total_distance(p, cities)
        if distance < min_distance:
            min_distance = distance
            min_path = p

    end_time = time.time()
    execution_time = end_time - start_time

    print("Best Path:", end='')
    for city in min_path:
        print(" ", city, end='')
    print()
    print("Minimum Distance:", min_distance)
    print("Execution Time:", execution_time, "seconds")

if __name__ == "__main__":
    nodes = [
        City(60, 200), City(180, 200), City(80, 180), City(140, 180), City(20, 160),
        City(100, 160), City(200, 160), City(140, 140), City(40, 120), City(100, 120)
    ]

    naive_tsp(nodes)


動態規劃 //Execute:0.2-0.4s

In [ ]:
import math
import itertools
import time

class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y

def distance(city1, city2):
    return math.sqrt((city1.x - city2.x) ** 2 + (city1.y - city2.y) ** 2)

def tsp(nodes):
    n = len(nodes)
    dp = [[float('inf')] * n for _ in range(1 << n)]
    path = [[-1] * n for _ in range(1 << n)]

    # Base case: if there's only one node
    dp[1][0] = 0

    for mask in range(1, 1 << n):
        for i in range(n):
            if mask & (1 << i):
                for j in range(n):
                    if mask & (1 << j):
                        if dp[mask ^ (1 << i)][j] + distance(nodes[j], nodes[i]) < dp[mask][i]:
                            dp[mask][i] = dp[mask ^ (1 << i)][j] + distance(nodes[j], nodes[i])
                            path[mask][i] = j

    min_distance = float('inf')
    last_node = -1
    for i in range(n):
        if dp[(1 << n) - 1][i] + distance(nodes[i], nodes[0]) < min_distance:
            min_distance = dp[(1 << n) - 1][i] + distance(nodes[i], nodes[0])
            last_node = i

    for mask in range(1 << n):
        print([i for i in range(n) if mask & (1 << i)], end=" ")
        print(" Distance: ", dp[mask][last_node] + distance(nodes[last_node], nodes[0]))

    return min_distance

if __name__ == "__main__":
    nodes = [
        City(60, 200), City(180, 200), City(80, 180), City(140, 180), City(20, 160),
        City(100, 160), City(200, 160), City(140, 140), City(40, 120), City(100, 120)
    ]

    start_time = time.time()
    min_distance = tsp(nodes)
    end_time = time.time()

    print("Minimum Distance:", min_distance)
    print("Elapsed time:", end_time - start_time, "s")


爬山演算法

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

def euclidean_distance(point1, point2):
    """計算兩點之間的歐氏距離"""
    return np.linalg.norm(point1 - point2)

def total_distance(points, order):
    """計算路徑的總距離"""
    distance = 0
    for i in range(len(order) - 1):
        distance += euclidean_distance(points[order[i]], points[order[i+1]])
    distance += euclidean_distance(points[order[-1]], points[order[0]])  # 回到起點
    return distance

def climb_hill(points):
    """爬山演算法求解TSP"""
    num_cities = len(points)
    order = np.arange(num_cities)  # 初始路徑順序
    np.random.shuffle(order)  # 隨機初始化路徑順序

    best_order = order.copy()
    best_distance = total_distance(points, best_order)
    start_time = time.time()

    while True:
        improved = False
        for i in range(num_cities):
            for j in range(i + 1, num_cities):
                new_order = best_order.copy()
                new_order[i], new_order[j] = new_order[j], new_order[i]  # 交換兩個城市的位置
                new_distance = total_distance(points, new_order)
                if new_distance < best_distance:
                    best_order = new_order
                    best_distance = new_distance
                    improved = True
        if not improved:
            break

    end_time = time.time()
    execution_time = end_time - start_time

    return best_order, best_distance, execution_time

# 定義城市坐標
points = np.array([
    [60, 200], [180, 200], [80, 180], [140, 180],
    [20, 160], [100, 160], [200, 160], [140, 140],
    [40, 120], [100, 120]
])

# 使用爬山演算法求解TSP
best_order, best_distance, execution_time = climb_hill(points)

# 顯示結果
print("最佳路徑順序:", best_order)
print("最佳路徑總距離:", best_distance)
print("計算時間:", execution_time, "秒")

# 繪製路徑圖表
plt.figure(figsize=(8, 6))
plt.scatter(points[:, 0], points[:, 1], c='blue', label='Cities')
for i in range(len(best_order) - 1):
    plt.plot([points[best_order[i]][0], points[best_order[i+1]][0]],
             [points[best_order[i]][1], points[best_order[i+1]][1]],
             c='red', linewidth=1)
plt.plot([points[best_order[-1]][0], points[best_order[0]][0]],
         [points[best_order[-1]][1], points[best_order[0]][1]],
         c='red', linewidth=1, label='Best Path')
plt.scatter(points[best_order[0]][0], points[best_order[0]][1], c='green', label='Start/End')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Traveling Salesman Problem - Hill Climbing')
plt.legend()
plt.grid(True)
plt.show()

退火演算法.py

In [ ]:
import numpy as np
import random
import time
import matplotlib.pyplot as plt  # 引入 Matplotlib 用於繪圖

def distance(points, order):
    """
    計算給定順序的路徑總距離
    """
    total_distance = 0
    for i in range(len(order) - 1):
        total_distance += np.linalg.norm(points[order[i]] - points[order[i + 1]])
    total_distance += np.linalg.norm(points[order[-1]] - points[order[0]])
    return total_distance

def acceptance_probability(old_cost, new_cost, temperature):
    """
    接受新解的概率
    """
    if new_cost < old_cost:
        return 1.0
    return np.exp((old_cost - new_cost) / temperature)

def simulated_annealing(points, initial_temperature=1000, cooling_rate=0.003, num_iter=10000):
    """
    模擬退火算法
    """
    num_cities = len(points)
    current_order = list(range(num_cities))
    random.shuffle(current_order)
    current_cost = distance(points, current_order)

    best_order = current_order.copy()
    best_cost = current_cost

    temperature = initial_temperature

    start_time = time.time()

    for _ in range(num_iter):
        new_order = current_order.copy()
        i, j = random.sample(range(num_cities), 2)
        new_order[i], new_order[j] = new_order[j], new_order[i]

        new_cost = distance(points, new_order)

        if random.random() < acceptance_probability(current_cost, new_cost, temperature):
            current_order = new_order
            current_cost = new_cost

            if current_cost < best_cost:
                best_order = current_order.copy()
                best_cost = current_cost

        temperature *= 1 - cooling_rate

    end_time = time.time()
    execution_time = end_time - start_time

    return best_order, best_cost, execution_time

if __name__ == "__main__":
    # 使用者輸入城市的坐標
    points = np.array([
        [60, 200], [180, 200], [80, 180], [140, 180],
        [20, 160], [100, 160], [200, 160], [140, 140],
        [40, 120], [100, 120]
    ])

    # 使用退火算法找到最佳路徑
    best_order, best_cost, execution_time = simulated_annealing(points)

    # 輸出結果
    print("最佳路徑順序：", best_order)
    print("最佳路徑總距離：", best_cost)
    print("計算時間：", execution_time)

    # 繪製圖表
plt.figure(figsize=(8, 6))
plt.scatter(points[:,0], points[:,1], c='blue', label='城市')  # 繪製城市
plt.plot(points[best_order][:,0], points[best_order][:,1], c='red', linestyle='-', linewidth=1.5, label='最佳路徑')  # 繪製最佳路徑
# 加入回到起點的路線
plt.plot([points[best_order[-1]][0], points[best_order[0]][0]], [points[best_order[-1]][1], points[best_order[0]][1]], c='red', linestyle='-', linewidth=1.5)
plt.scatter(points[best_order[0]][0], points[best_order[0]][1], c='green', marker='o', label='start')  # 標記起點
plt.xlabel('x')
plt.ylabel('y')
plt.title('best Trip')
plt.legend()
plt.grid(True)
plt.show()

爬山演算法 vs 退火演算法 退火較差

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import math
import time

class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y

def distance(city1, city2):
    return np.sqrt((city1.x - city2.x) ** 2 + (city1.y - city2.y) ** 2)

def total_distance(path):
    total = 0
    for i in range(len(path) - 1):
        total += distance(path[i], path[i + 1])
    total += distance(path[-1], path[0])  # 回到起點
    return total

def hill_climbing_tsp(nodes):
    current_path = nodes.copy()
    current_distance = total_distance(current_path)

    improved = True
    while improved:
        improved = False
        for i in range(len(nodes) - 1):
            for j in range(i + 1, len(nodes)):
                current_path[i], current_path[j] = current_path[j], current_path[i]
                new_distance = total_distance(current_path)
                if new_distance < current_distance:
                    current_distance = new_distance
                    improved = True
                else:
                    current_path[i], current_path[j] = current_path[j], current_path[i]
    return current_path

def simulated_annealing_tsp(nodes, initial_temperature, cooling_rate, iterations):
    current_path = nodes.copy()
    best_path = current_path.copy()
    current_distance = total_distance(current_path)
    best_distance = current_distance

    for i in range(iterations):
        temperature = initial_temperature / (1 + cooling_rate * i)
        random_index1 = random.randint(0, len(nodes) - 1)
        random_index2 = random.randint(0, len(nodes) - 1)
        current_path[random_index1], current_path[random_index2] = current_path[random_index2], current_path[random_index1]

        new_distance = total_distance(current_path)
        delta = new_distance - current_distance

        if delta < 0 or math.exp(-delta / temperature) > random.random():
            current_distance = new_distance
            if current_distance < best_distance:
                best_path = current_path.copy()
                best_distance = current_distance
        else:
            current_path[random_index1], current_path[random_index2] = current_path[random_index2], current_path[random_index1]

    return best_path

def plot_cities(cities, title):
    x = [city.x for city in cities]
    y = [city.y for city in cities]
    plt.figure(figsize=(6, 6))
    plt.scatter(x, y)
    plt.title(title)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.grid(True)
    plt.show()

def plot_route(cities, route, title):
    x = [city.x for city in cities]
    y = [city.y for city in cities]
    plt.figure(figsize=(6, 6))
    plt.plot(x, y, 'bo-')
    for i, city in enumerate(route):
        if i == len(route) - 1:
            plt.plot([city.x, route[0].x], [city.y, route[0].y], 'bo-')
        else:
            plt.plot([city.x, route[i+1].x], [city.y, route[i+1].y], 'bo-')
    plt.title(title)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.grid(True)
    plt.show()

def main():
    nodes = [
      City(60, 200), City(180, 200), City(80, 180), City(140, 180), City(20, 160),
      City(100, 160), City(200, 160), City(140, 140), City(40, 120), City(100, 120),
      City(60, 100), City(180, 100), City(80, 80), City(140, 80), City(20, 60),
      City(100, 60), City(200, 60), City(140, 40), City(40, 20), City(100, 20),
      City(60, 300), City(180, 300), City(80, 280), City(140, 280), City(20, 260),
      City(100, 260), City(200, 260), City(140, 240), City(40, 220), City(100, 220)
  ]




    initial_temperature = 1000
    cooling_rate = 0.003
    iterations = 10000

    # 爬山演算法
    start_time = time.time()
    shortest_path_hill_climbing = hill_climbing_tsp(nodes)
    end_time = time.time()
    print("Shortest Path (Hill Climbing):", [vars(city) for city in shortest_path_hill_climbing])
    print("Total Distance (Hill Climbing):", total_distance(shortest_path_hill_climbing))
    print("Time (Hill Climbing):", end_time - start_time, "seconds")
    plot_route(nodes, shortest_path_hill_climbing, "Shortest Path (Hill Climbing)")

    # 模擬退火算法
    start_time = time.time()
    shortest_path_simulated_annealing = simulated_annealing_tsp(nodes, initial_temperature, cooling_rate, iterations)
    end_time = time.time()
    print("Shortest Path (Simulated Annealing):", [vars(city) for city in shortest_path_simulated_annealing])
    print("Total Distance (Simulated Annealing):", total_distance(shortest_path_simulated_annealing))
    print("Time (Simulated Annealing):", end_time - start_time, "seconds")
    plot_route(nodes, shortest_path_simulated_annealing, "Shortest Path (Simulated Annealing)")

if __name__ == "__main__":
    main()
